<a href="https://colab.research.google.com/github/vnaticzhock/Bank-Fraud-Detector/blob/main/LSTM_Closing_price_only.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install statsmodels --upgrade
# !pip install pmdarima
!pip install arch

In [ ]:
import torch
import torch.nn as nn
import pandas as pd
from statsmodels.tsa.stattools import adfuller
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.stats.diagnostic import het_arch
import statsmodels.graphics.tsaplots as sgt

In [ ]:
def timeFormat(time: str) -> str:
  tmp = time.split('/')
  tmp[0] = str(int(tmp[0]) + 1911)
  return '-'.join(tmp)

df = pd.read_csv('/content/日收盤價及成交量等.csv')
df['Unnamed: 0'] = df['Unnamed: 0'].apply(lambda x: timeFormat(x))
df.index = pd.Index(df['Unnamed: 0'].copy())
df = df.sort_index()
dta = pd.concat([df['Close-Price'].pct_change() * 100, df['Close-Price']], axis=1).dropna()
dta.columns = ['日報酬率(%)', '收盤價']
dta.index.name = '年月日'

print(len(dta))
dta.tail(3)

In [ ]:
price_p_value = adfuller(dta['日報酬率(%)'])[1]
print(price_p_value)
if price_p_value > 0.05:
    print('Nonstationary')
else:
    print('Stationary')

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

roi = dta['日報酬率(%)'].values

In [ ]:
from torch.utils.data import Dataset, DataLoader
import numpy as np

class StockDataset(Dataset):
  def __init__(self, x, df, mode='train'):
    self.mode = mode

    length = int(len(x) / 10 * 9)

    if mode == 'train':
      indices = [i for i in range(0, length)]
    elif mode == 'dev':
      indices = [i for i in range(length - 90, len(x))]
    else: ## test
      indices = [i for i in range(len(x))]

    self.data = torch.FloatTensor(np.array(x).astype(float)[indices])
    self.dataframe = df.iloc[indices]

    self.target = []
    for i in range(90, len(self.data)):
      self.target.append(self.data[i])

    self.target = torch.FloatTensor(np.array(self.target).astype(float))

    self.dim = self.data.shape[1]

  def __getitem__(self, index):
      return torch.FloatTensor(self.data[index : index + 90]), torch.FloatTensor(self.target[index])

  def __len__(self):
      return len(self.target)

In [ ]:
# x = scaler.fit_transform(close_price)
x = roi.reshape(-1, 1)

stockDataset = StockDataset(list(x), dta, 'train')
train_set = DataLoader(
        stockDataset, 32,
        shuffle=False, drop_last=False,
        num_workers=0, pin_memory=True) 

devDataset = StockDataset(list(x), dta, 'dev')
dev_set = DataLoader(
        devDataset, 32,
        shuffle=False, drop_last=False,
        num_workers=0, pin_memory=True)

In [ ]:
from collections import Counter 

print(len(stockDataset), len(devDataset))

In [ ]:
import torch.nn.functional as F
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class LSTM_Attention(nn.Module):
  def __init__(self):

    self.embedding_dim = stockDataset.dim
    self.n_hidden = 16

    super(LSTM_Attention, self).__init__()
    self.query = torch.randn(self.n_hidden).to(device)
    self.lstm = nn.LSTM(self.embedding_dim, self.n_hidden)
    self.linear = nn.Linear(self.n_hidden, 32)
    self.activation = nn.ReLU()
    self.out = nn.Linear(32, 1)
    self.dropout = nn.Dropout(p=0.3)


  def attention_net(self, lstm_output):

    attn_weights = (lstm_output * self.query.view(1,1, self.n_hidden)).sum(2)
    soft_attn_weights = F.softmax(attn_weights, 1)

    context = (lstm_output * soft_attn_weights.view(-1, 90, 1)).sum(1)

    return context, soft_attn_weights

  def forward(self, x):
    '''
    :param X: [Batch size, 天數, 特徵數量]
    '''
    x = x.transpose(0, 1)
    output, (final_hidden_state, final_cell_state) = self.lstm(x)
    ## output: [20, 8, 10] = [天數, batch_size, embedding]
    output = output.transpose(0, 1)
    final_hidden_state = final_hidden_state.transpose(0, 1)
    final_cell_state = final_cell_state.transpose(0, 1)

    attn_output, attention = self.attention_net(output)
    
    return self.out(self.activation(self.linear(self.dropout(attn_output)))), attention

class LSTM(nn.Module):
  def __init__(self):

    embedding_dim = stockDataset.dim
    n_hidden = 10

    super(LSTM, self).__init__()
    self.lstm = nn.LSTM(embedding_dim, n_hidden, num_layers=2)
    self.linear = nn.Linear(n_hidden * 2, 32)
    self.activation = nn.ReLU()
    self.out = nn.Linear(32, 1)
    self.dropout = nn.Dropout(p=0.3)

    self.net = nn.Sequential(
        self.dropout,
        self.linear,
        self.activation,
        self.out,
    )

  def forward(self, x):
    '''
    :param X: [Batch size, 天數, 特徵數量]
    '''
    # print(x.size())
    x = x.transpose(0, 1)
    output, (final_hidden_state, final_cell_state) = self.lstm(x)
    # print(final_hidden_state.size())

    return self.net(torch.flatten(final_hidden_state.transpose(0, 1), start_dim=1)), 0

class BiLSTM(nn.Module):
  def __init__(self):

    embedding_dim = stockDataset.dim
    n_hidden = 10

    super(BiLSTM, self).__init__()
    self.lstm = nn.LSTM(embedding_dim, n_hidden, bidirectional=True)
    self.linear = nn.Linear(n_hidden * 2, 32)
    self.activation = nn.ReLU()
    self.out = nn.Linear(32, 1)
    self.dropout = nn.Dropout(p=0.3)

    self.net = nn.Sequential(
        self.dropout,
        self.linear,
        self.activation,
        self.out,
    )

  def forward(self, x):
    '''
    :param X: [Batch size, 天數, 特徵數量]
    '''
    x = x.transpose(0, 1)
    output, (final_hidden_state, final_cell_state) = self.lstm(x)

    return self.net(torch.flatten(final_hidden_state.transpose(0, 1), start_dim=1)), 0
        

model = BiLSTM().to(device)
loss_fn = nn.MSELoss(reduction='mean').to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
from tqdm import tqdm
from sklearn.metrics import f1_score

min_mse = 1000.
for epoch in tqdm(range(300)):

  model.train()

  counter_tr = 0

  for x, y in train_set:
    optimizer.zero_grad()

    x, y = x.to(device), y.to(device)
    pred, attention = model(x)
    loss = loss_fn(pred, y)

    counter_tr += float(loss.cpu().detach()) * len(y)

    loss.backward()
    optimizer.step()

  ## validating

  counter = 0

  model.eval()
  for x, y in dev_set:
    with torch.no_grad():
      x, y = x.to(device), y.to(device)
      pred, attention = model(x)
      loss = loss_fn(pred, y)
      
      counter += float(loss.cpu().detach()) * len(y)

  dev_mse = counter / len(devDataset)
  train_mse = counter_tr / len(stockDataset)
  if (epoch + 1) % 15 == 0:
    print(f'\n Epoch: {epoch+1},\n dev_loss: {dev_mse},\n train_loss: {train_mse}')
    # print(attention)

  if epoch > 120 and dev_mse < min_mse:
    # Save model if your model improved
    min_mse = dev_mse
    torch.save(model.state_dict(), './model.pth')
    print(f'\n Epoch: {epoch+1},\n lowest now: {min_mse}')


'''
BiLSTM
Epoch: 192,
lowest now: 0.00011559145147629985

Epoch: 282,
 lowest now: 9.559588342029005e-05

LSTM stacked
Epoch: 238,
lowest now: 0.0001152252194570434

Attention
Epoch: 235,
lowest now: 0.00011564536036916472
'''

In [ ]:
model = BiLSTM().to(device)
ckpt = torch.load('./model.pth', map_location='cpu')  # Load your best model
model.load_state_dict(ckpt)

x = roi.reshape(-1, 1)

# totalDataset = StockDataset(list(x), df, 'total')
# tt_set = DataLoader(
#         totalDataset, 32,
#         shuffle=False, drop_last=False,
#         num_workers=0, pin_memory=True)

truth = []
preds = []

for x, y in train_set:
    with torch.no_grad():
      model.eval()
      x, y = x.to(device), y.to(device)
      pred, attention = model(x)

      for i in range(len(y)):
        truth.append(float(y[i].cpu().detach()))
        preds.append(float(pred[i].cpu().detach()))

residual = np.array(truth) - np.array(preds)

truth = []
preds = []

for x, y in dev_set:
    with torch.no_grad():
      model.eval()
      x, y = x.to(device), y.to(device)
      pred, attention = model(x)

      for i in range(len(y)):
        truth.append(float(y[i].cpu().detach()))
        preds.append(float(pred[i].cpu().detach()))


In [ ]:
print(truth)
print(devDataset.dataframe.iloc[90])
## 預測的第一天是從 2021-05-06 開始，從2020-12-15開始拿90天的資料預測下一天

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(len(truth)), truth, label='Fact')
plt.plot(range(len(preds)), preds, label='Predict')
plt.legend()
plt.show()

In [ ]:
white_noise = acorr_ljungbox(residual, lags = [10], return_df=True)
print(white_noise)

LM_pvalue = het_arch(residual, ddof = 4)[1]
print('LM-test-Pvalue:', '{:.5f}'.format(LM_pvalue))

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (18,5))

sgt.plot_acf(residual**2, zero = False, lags = 40, ax=ax[0])
sgt.plot_pacf(residual**2, zero = False, lags = 40, ax=ax[1])
plt.show()

In [ ]:
# plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']

date = devDataset.dataframe.index[-len(devDataset):]

res = pd.DataFrame([truth, preds]).transpose()
res.columns = ['日報酬率(%)', 'LSTM ROI']

plt.figure(figsize=(15,8))

plt.plot(res['日報酬率(%)'])
plt.plot(res['LSTM ROI'])

plt.legend(('Truth', 'LSTM'), fontsize=16)

In [ ]:
print(date)

In [ ]:
from arch import arch_model

## 最難的地方！😭
truth = []
preds = []

x = roi.reshape(-1, 1)

totalDataset = StockDataset(list(x), dta, 'total')
tt_set = DataLoader(
        totalDataset, 32,
        shuffle=False, drop_last=False,
        num_workers=0, pin_memory=True)

for x, y in tt_set:
    with torch.no_grad():
      model.eval()
      x, y = x.to(device), y.to(device)
      pred, attention = model(x)

      for i in range(len(y)):
        truth.append(float(y[i].cpu().detach()))
        preds.append(float(pred[i].cpu().detach()))

residual = np.array(truth) - np.array(preds)

garch_forecast = []
for i in range(len(devDataset)):
    # print(f'from 0 to {-(len(devDataset)-i)}')
    train = residual[:-(len(devDataset)-i)]
    model = arch_model(train, vol = 'GARCH', p = 1, q = 1)
    garch_fit = model.fit()
    prediction = garch_fit.forecast(horizon=1)
    garch_forecast.append(np.sqrt(prediction.variance.values[-1:][0]))

In [ ]:
print(len(res), len(garch_forecast))

In [ ]:
## 得到 test 的日報酬率(len=len(devDataset)); LSTM 預測的報酬率
res['GARCH預測波動度'] = (garch_forecast)
res['預測區間上限'] = res['LSTM ROI'] + res['GARCH預測波動度']
res['預測區間下限'] = res['LSTM ROI'] - res['GARCH預測波動度']

In [ ]:
first_price = devDataset.dataframe['收盤價'][date[0]] / (1+res['日報酬率(%)'][0]*0.01)
print(date[0], devDataset.dataframe['收盤價'][date[0]], res['日報酬率(%)'][0])

In [ ]:
# 計算第一期預測
res['LSTM 預測價格'] = first_price * (1 + res['LSTM ROI']*0.01)
res['預測價格區間上限'] = first_price * (1 + res['預測區間上限']*0.01)
res['預測價格區間下限'] = first_price * (1 + res['預測區間下限']*0.01)

# 計算剩餘預測區間
for i in range(1, len(res)):
        res['LSTM 預測價格'][i] = res['LSTM 預測價格'][i-1] * (1 + res['LSTM ROI'][i]*0.01)
        res['預測價格區間上限'][i] = res['預測價格區間上限'][i-1] * (1 + res['預測區間上限'][i]*0.01)
        res['預測價格區間下限'][i] = res['預測價格區間下限'][i-1] * (1 + res['預測區間下限'][i]*0.01)
# 計算區間均價
res['預測平均價格'] = (res['預測價格區間上限'] + res['預測價格區間下限']) / 2

In [ ]:
res['收盤價'] = devDataset.dataframe['收盤價'][date].values

In [ ]:
res

In [ ]:
plt.figure(figsize=(15,8))

plt.plot(res['收盤價'][:30], color = 'b')
plt.plot(res['LSTM 預測價格'][:30], color = 'r')
plt.plot(res['預測平均價格'][:30], color = 'r', linestyle='dashed')
plt.plot(res['預測價格區間上限'][:30], color = 'g')
plt.plot(res['預測價格區間下限'][:30], color = 'g')


plt.legend(('Real', 'LSTM Anticipate', 'Mean', 'Upper bound', 'Downer bound'), fontsize=16)
# plt.legend(('Real', 'LSTM Anticipate'), fontsize=16)

In [ ]:
garch_std_resid = pd.Series(garch_fit.resid / garch_fit.conditional_volatility)
white_noise_garch = acorr_ljungbox(garch_std_resid, lags = [10], return_df=True)
white_noise_garch